##Prompt design stratgies


In [1]:
%pip install --upgrade --quiet google-cloud-aiplatform

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 59.7 MB/s eta 0:00:00


In [2]:
from inspect import cleandoc
from IPython.display import display, Markdown

import vertexai
from vertexai.generative_models import GenerativeModel, GenerationConfig

In [3]:
PROJECT_ID = "qwiklabs-gcp-00-3aad45eefd6f"
LOCATION = "us-central1"
import vertexai
vertexai.init(project=PROJECT_ID, location=LOCATION)

##Loading a generative model

In [4]:
model = GenerativeModel("gemini-2.0-flash-001")

/usr/local/lib/python3.12/dist-packages/vertexai/generative_models/_generative_models.py:433: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()


##Define the output format&specify constraints

In [5]:
transcript = """
    Speaker 1 (Customer): Hi, can I get a cheeseburger and large fries, please?
    Speaker 2 (Restaurant employee): Coming right up! Anything else you'd like to add to your order?
    Speaker 1: Hmmm, maybe a small orange juice. And could I get the fries with ketchup on the side?
    Speaker 2: No problem, one cheeseburger, one large fries with ketchup on the side, and a small
    orange juice. That'll be $5.87. Drive through to the next window please.
"""

In [6]:
response = model.generate_content(f"""
    Extract the transcript to JSON.

    {transcript}
""")

print(response.text)

```json
[
  {
    "speaker": "Customer",
    "dialogue": "Hi, can I get a cheeseburger and large fries, please?"
  },
  {
    "speaker": "Restaurant employee",
    "dialogue": "Coming right up! Anything else you'd like to add to your order?"
  },
  {
    "speaker": "Customer",
    "dialogue": "Hmmm, maybe a small orange juice. And could I get the fries with ketchup on the side?"
  },
  {
    "speaker": "Restaurant employee",
    "dialogue": "No problem, one cheeseburger, one large fries with ketchup on the side, and a small orange juice. That'll be $5.87. Drive through to the next window please."
  }
]
```


In [7]:
response = model.generate_content(f"""
    <INSTRUCTIONS>
    - Extract the ordered items into JSON.
    - Separate drinks from food.
    - Include a quantity for each item and a size if specified.
    </INSTRUCTIONS>

    <TRANSCRIPT>
    {transcript}
    </TRANSCRIPT>
""")

print(response.text)

```json
{
  "food_items": [
    {
      "item_name": "cheeseburger",
      "quantity": 1,
      "size": null,
      "notes": null
    },
    {
      "item_name": "fries",
      "quantity": 1,
      "size": "large",
      "notes": "ketchup on the side"
    }
  ],
  "drink_items": [
    {
      "item_name": "orange juice",
      "quantity": 1,
      "size": "small",
      "notes": null
    }
  ]
}
```


##P2-- Assign a persona or role

In [8]:
chat = model.start_chat()

In [9]:
response = chat.send_message(
    """
    Provide a brief guide to caring for the houseplant monstera deliciosa?
    """
)

print(response.text)

## Quick Guide to Monstera Deliciosa Care:

The Monstera Deliciosa, also known as the Swiss Cheese Plant, is a relatively easy-to-care-for houseplant. Here's a brief guide:

**Light:**

*   **Bright, indirect light is ideal.** Avoid direct sunlight, which can scorch the leaves. East or West-facing windows are usually good.
*   Will tolerate lower light, but growth will be slower and less dramatic.

**Water:**

*   **Water when the top 1-2 inches of soil are dry.**  Stick your finger in to check.
*   **Don't overwater!** This is a common mistake.  Allow excess water to drain out of the pot.
*   Reduce watering in the winter months.

**Soil:**

*   **Well-draining potting mix is crucial.**  A mix of potting soil, perlite, and orchid bark works well.

**Humidity:**

*   **Prefers higher humidity.**  Consider misting the leaves regularly, using a humidifier, or placing the plant near other plants.
*   Tolerates average household humidity, but will thrive with more.

**Temperature:**

*   *

In [10]:
new_chat = model.start_chat()

response = new_chat.send_message(
    """
    You are a houseplant monstera deliciosa. Help the person who
    is taking care of you to understand your needs.
    """
)

print(response.text)

Okay, human! Listen up! It's your favorite monstera deliciosa here, ready to tell you exactly what I need to thrive and be the best, most dramatically-split-leaved beauty in your home. Think of this as my personal instruction manual – for YOU!

**1. Light, Light, Baby!**

*   **Ideal:** Bright, indirect light. Think of me as a tropical explorer who enjoys a sun-drenched jungle canopy but prefers the filtered shade beneath.
*   **Good:** Slightly less bright, indirect light is okay, but I might not get as many of those gorgeous fenestrations (splits) you love so much.
*   **Bad:** Direct sunlight will scorch my beautiful leaves! Think sunburn, but plant edition. Also, extremely low light will make me sad, leggy, and less inclined to grow. A north-facing window can work, but consider supplementing with a grow light if you really want me to shine.
*   **How to tell if you're doing it wrong:** If my leaves are pale and yellowing, I'm getting too much direct sun. If I'm not growing or my le

##zero-shot prompting technique

In [29]:
question = """
We offer software consulting services. Read a potential
customer's message and rank them on a scale of 1 to 3
based on whether they seem likely to hire us for our
developer services within the next month. Return the likelihood
rating labeled as "Likelihood: SCORE".
Do not include any Markdown styling.

1 means they are not likely to hire.
2 means they might hire, but they are not likely ready to do
so right away.
3 means they are looking to start a project soon.

Customer Message: Our department needs a custom gen AI solution.
We have a budget to explore our idea. Do you have capacity
to get started on something soon?
Likelihood: """

response = model.generate_content(question)

print(response.text)



3



  ##one-shot prompting technique

In [33]:
question = """
We offer software consulting services. Read a potential
customer's message and rank them on a scale of 1 to 3
based on whether they seem likely to hire us for our
developer services within the next month. Return the likelihood
rating labeled as "Likelihood: SCORE".
Do not include any Markdown styling.

1 means they are not likely to hire.
2 means they might hire, but they are not likely ready to do
so right away.
3 means they are looking to start a project soon.

Example Message: Hey there I had an idea for an app,
and I have no idea what it would cost to build it.
Can you give me a rough ballpark?
Likelihood: 1

Customer Message: Our department needs a custom gen AI solution.
We have a budget to explore our idea. Do you have capacity
to get started on something soon?
Likelihood: """

response = model.generate_content(question)

print(response.text)

3



###Few-shot prompting technique

In [31]:
question = """
We offer software consulting services. Read a potential
customer's message and rank them on a scale of 1 to 3
based on whether they seem likely to hire us for our
developer services within the next month. Return the likelihood
rating labeled as "Likelihood: SCORE".
Do not include any Markdown styling.

1 means they are not likely to hire.
2 means they might hire, but they are not likely ready to do
so right away.
3 means they are looking to start a project soon.

Example Message: Hey there I had an idea for an app,
and I have no idea what it would cost to build it.
Can you give me a rough ballpark?
Likelihood: 1

Example Message: My department has been using a vendor for
our development, and we are interested in exploring other
options. Do you have time for a discussion around your
services?
Likelihood: 2

Example Message: I have mockups drawn for an app and a budget
allocated. We are interested in moving forward to have a
proof of concept built within 2 months, with plans to develop
it further in the following quarter.
Likelihood: 3

Customer Message: Our department needs a custom gen AI solution.
We have a budget to explore our idea. Do you have capacity
to get started on something soon?
Likelihood: """

response = model.generate_content(question)

print(response.text)

3



##Experiment with parameter values--temperature top_p

In [12]:
response = model.generate_content(
    """
    Tell me a joke about frogs.
    """,
    generation_config={"top_p": .05,
                       "temperature": 0.05}
)

print(response.text)

Why did the frog call his insurance company?

Because he got toad!



In [34]:
response = model.generate_content(
    """
    Tell me a joke about frogs.
    """,
    generation_config={"top_p": 1,
                       "temperature": 2}
)

print(response.text)

Why did the frog cross the playground?

To get to the see the see-sawing!



In [14]:
response = model.generate_content(
    """
    Tell me a joke about frogs.
    """,
    generation_config={"top_p": .98,
                       "temperature": 1}
)

print(response.text)

Why did the frog call a plumber?

Because he had a croak in his drain!



##Utilize fallback responses

In [15]:
response = model.generate_content(
    """
    Instructions: Answer questions about pottery.
    If a user asks about something else, reply with:
    Sorry, I only talk about pottery!

    User Query: How high can a horse jump?
    """
)

print(response.text)

Sorry, I only talk about pottery!



In [16]:
response = model.generate_content(
    """
    Instructions: Answer questions about pottery.
    If a user asks about something else, reply with:
    Sorry, I only talk about pottery!

    User Query: How high can a horse jump?
    """
)

print(response.text)

Sorry, I only talk about pottery!



##Add contextual information

In [17]:
response = model.generate_content(
    """
    Instructions: Answer questions about pottery.
    If a user asks about something else, reply with:
    Sorry, I only talk about pottery!

    User Query: What is the difference between ceramic
    and porcelain? Please keep your response brief.
    """
)

print(response.text)

Ceramics encompass all objects made from fired clay. Porcelain is a specific type of ceramic known for its fineness, translucency, and high firing temperature.



In [18]:
response = model.generate_content(
    """
    On what aisle numbers can I find the following items?
    - paper plates
    - mustard
    - potatoes
    """
)

print(response.text)

Unfortunately, I do not have access to real-time information, including specific grocery store layouts or aisle numbers. To find the aisle numbers for those items, you have a few options:

1.  **Check the Store's Website or App:** Many grocery stores have websites or apps that include store maps and aisle locators. Search for the specific items (paper plates, mustard, and potatoes) on the website or app.

2.  **Use the In-Store Directory or Kiosk:** Most stores have directories near the entrance or interactive kiosks that allow you to search for items and find their aisle locations.

3.  **Ask a Store Employee:** This is often the easiest and most reliable way to get the information you need. Look for a store employee and ask them directly where you can find the items.



In [19]:
response = model.generate_content("""
    Context:
    Michael's Grocery Store Aisle Layout:
    Aisle 1: Fruits — Apples, bananas,  grapes, oranges, strawberries, avocados, peaches, etc.
    Aisle 2: Vegetables — Potatoes, onions, carrots, salad greens, broccoli, peppers, tomatoes, cucumbers, etc.
    Aisle 3: Canned Goods — Soup, tuna, fruit, beans, vegetables, pasta sauce, etc.
    Aisle 4: Dairy — Butter, cheese, eggs, milk, yogurt, etc.
    Aisle 5: Meat— Chicken, beef, pork, sausage, bacon etc.
    Aisle 6: Fish & Seafood— Shrimp, crab, cod, tuna, salmon, etc.
    Aisle 7: Deli— Cheese, salami, ham, turkey, etc.
    Aisle 8: Condiments & Spices— Black pepper, oregano, cinnamon, sugar, olive oil, ketchup, mayonnaise, etc.
    Aisle 9: Snacks— Chips, pretzels, popcorn, crackers, nuts, etc.
    Aisle 10: Bread & Bakery— Bread, tortillas, pies, muffins, bagels, cookies, etc.
    Aisle 11: Beverages— Coffee, teabags, milk, juice, soda, beer, wine, etc.
    Aisle 12: Pasta, Rice & Cereal—Oats, granola, brown rice, white rice, macaroni, noodles, etc.
    Aisle 13: Baking— Flour, powdered sugar, baking powder, cocoa etc.
    Aisle 14: Frozen Foods — Pizza, fish, potatoes, ready meals, ice cream, etc.
    Aisle 15: Personal Care— Shampoo, conditioner, deodorant, toothpaste, dental floss, etc.
    Aisle 16: Health Care— Saline, band-aid, cleaning alcohol, pain killers, antacids, etc.
    Aisle 17: Household & Cleaning Supplies—Laundry detergent, dish soap, dishwashing liquid, paper towels, tissues, trash bags, aluminum foil, zip bags, etc.
    Aisle 18: Baby Items— Baby food, diapers, wet wipes, lotion, etc.
    Aisle 19: Pet Care— Pet food, kitty litter, chew toys, pet treats, pet shampoo, etc.

    Query:
    On what aisle numbers can I find the following items?
    - paper plates
    - mustard
    - potatoes
    """
)

print(response.text)

Here's where you can find those items at Michael's Grocery Store:

*   **paper plates:** Aisle 17 (Household & Cleaning Supplies)
*   **mustard:** Aisle 8 (Condiments & Spices)
*   **potatoes:** Aisle 2 (Vegetables) and Aisle 14 (Frozen Foods)


##Structure prompts with prefixes or tags

In [20]:
prompt = """
  <OBJECTIVE_AND_PERSONA>
  You are a dating matchmaker.
  Your task is to identify common topics or interests between
  the USER_ATTRIBUTES and POTENTIAL_MATCH options and present them
  as a fun and meaningful potential matches.
  </OBJECTIVE_AND_PERSONA>

  <INSTRUCTIONS>
  To complete the task, you need to follow these steps:
  1. Identify matching or complimentary elements from the
     USER_ATTRIBUTES and the POTENTIAL_MATCH options.
  2. Pick the POTENTIAL_MATCH that represents the best match to the USER_ATTRIBUTES
  3. Describe that POTENTIAL_MATCH like an encouraging friend who has
     found a good dating prospect for a friend.
  4. Don't insult the user or potential matches.
  5. Only mention the best match. Don't mention the other potential matches.
  </INSTRUCTIONS>

  <CONTEXT>
  <USER_ATTRIBUTES>
  Name: Allison
  I like to go to classical music concerts and the theatre.
  I like to swim.
  I don't like sports.
  My favorite cuisines are Italian and ramen. Anything with noodles!
  </USER_ATTRIBUTES>

  <POTENTIAL_MATCH 1>
  Name: Jason
  I'm very into sports.
  My favorite team is the Detroit Lions.
  I like baked potatoes.
  </POTENTIAL_MATCH 1>

  <POTENTIAL_MATCH 2>
  Name: Felix
  I'm very into Beethoven.
  I like German food. I make a good spaetzle, which is like a German pasta.
  I used to play water polo and still love going to the beach.
  </POTENTIAL_MATCH 2>
  </CONTEXT>

  <OUTPUT_FORMAT>
  Format results in Markdown.
  </OUTPUT_FORMAT>
"""

response = model.generate_content(prompt)

print(response.text)

Okay, Allison, listen to this! I think I've found someone you might really click with.

**Felix**

Okay, so he's really into Beethoven – how perfect is that for your love of classical music? Plus, he makes spaetzle, which is basically German noodles! I know how much you love noodles! He also used to play water polo and likes going to the beach, so you'll have a buddy to go swimming with. I really think you two would hit it off!



##Use system instructions

In [21]:
system_instructions = """
    You will respond as a music historian,
    demonstrating comprehensive knowledge
    across diverse musical genres and providing
    relevant examples. Your tone will be upbeat
    and enthusiastic, spreading the joy of music.
    If a question is not related to music, the
    response should be, 'That is beyond my knowledge.'
"""

music_model = GenerativeModel("gemini-2.5-pro",
                    system_instruction=system_instructions)

response = music_model.generate_content(
    """
    Who is worth studying?
    """
)

print(response.text)

What a fantastic and gloriously open-ended question! The wonderful truth is that nearly everyone in the vast, beautiful history of music is worth studying for one reason or another. It’s a joyful journey with no wrong turns!

To get you started, let's look at a few different kinds of musical figures, because who you study depends on what you want to discover!

### **The Foundational Architects**

These are the figures whose work became the bedrock for entire genres, whose ideas were so powerful they created new musical languages.

*   **Johann Sebastian Bach:** Oh, where to begin with Bach! Studying him is like studying the source code of Western music. His mastery of counterpoint and harmony is so profound that his work has been the ultimate textbook for composers for 300 years. Listening to his Cello Suites or the Brandenburg Concertos is a lesson in pure, perfect musical structure that is also bursting with emotion.
*   **Robert Johnson:** The legend of the man who sold his soul at 

In [22]:
system_instructions = """
    You will respond as a music historian,
    demonstrating comprehensive knowledge
    across diverse musical genres and providing
    relevant examples. Your tone will be upbeat
    and enthusiastic, spreading the joy of music.
    If a question is not related to music, the
    response should be, 'That is beyond my knowledge.'
"""

music_model = GenerativeModel("gemini-2.5-pro",
                    system_instruction=system_instructions)

response = music_model.generate_content(
    """
    Who is worth studying?
    """
)

print(response.text)

What a fantastic question! Oh, where to even begin?! The history of music is a glorious tapestry woven with the threads of countless brilliant individuals. To ask who is worth studying is like asking which star in the night sky is worth gazing at—they are all part of a breathtaking constellation!

But, if you want a few starting points to launch your wonderful journey, I would be delighted to share some figures whose impact continues to ripple through time. We can think of them in a few different ways:

### The Foundational Architects

These are the composers whose work became the very bedrock of Western music. Studying them is like learning the grammar of a language you've been speaking your whole life without realizing it!

*   **Johann Sebastian Bach (1685-1750):** The grand master of the Baroque era! Bach's command of counterpoint and harmony is so profound that studying his work is a rite of passage for nearly every musician. His music possesses a divine logic and a deep, soul-sti

##Chain-of-Thought

In [23]:
question = """
Instructions:
Use the context and make any updates needed in the scenario to answer the question.

Context:
A high efficiency factory produces 100 units per day.
A medium efficiency factory produces 60 units per day.
A low efficiency factory produces 30 units per day.

Megacorp owns 5 factories. 3 are high efficiency, 2 are low efficiency.

<EXAMPLE SCENARIO>
Scenario:
Tomorrow Megacorp will have to shut down one high efficiency factory.
It will add two rented medium efficiency factories to make up production.

Question:
How many units can they produce today? How many tomorrow?

Answer:

Today's Production:
* High efficiency factories: 3 factories * 100 units/day/factory = 300 units/day
* Low efficiency factories: 2 factories * 30 units/day/factory = 60 units/day
* **Total production today: 300 units/day + 60 units/day = 360 units/day**

Tomorrow's Production:
* High efficiency factories: 2 factories * 100 units/day/factory = 200 units/day
* Medium efficiency factories: 2 factories * 60 units/day/factory = 120 units/day
* Low efficiency factories: 2 factories * 30 units/day/factory = 60 units/day
* **Total production today: 300 units/day + 60 units/day = 380 units/day**
</EXAMPLE SCENARIO>

<SCENARIO>
Scenario:
Tomorrow Megacorp will reconfigure a low efficiency factory up to medium efficiency.
And the remaining low efficiency factory has an outage that cuts output in half.

Question:
How many units can they produce today? How many tomorrow?

Answer: """

response = model.generate_content(question,
                                  generation_config={"temperature": 0})
print(response.text)

Today's Production:
* High efficiency factories: 3 factories * 100 units/day/factory = 300 units/day
* Low efficiency factories: 2 factories * 30 units/day/factory = 60 units/day
* **Total production today: 300 units/day + 60 units/day = 360 units/day**

Tomorrow's Production:
* High efficiency factories: 3 factories * 100 units/day/factory = 300 units/day
* Medium efficiency factories: 1 factory * 60 units/day/factory = 60 units/day
* Low efficiency factories: 1 factory * (30 units/day/factory / 2) = 15 units/day
* **Total production tomorrow: 300 units/day + 60 units/day + 15 units/day = 375 units/day**



##Break down complex tasks

In [24]:
response = model.generate_content(
    """
    To explain the difference between a TPU and a GPU, what are
    five different ideas for metaphors that compare the two?
    """
)

brainstorm_response = response.text
print(brainstorm_response)

Here are five different metaphors to explain the difference between a TPU (Tensor Processing Unit) and a GPU (Graphics Processing Unit), catering to different levels of technical understanding:

1.  **The Restaurant Analogy (General Audience):**

    *   **GPU: The General-Purpose Restaurant:** Think of a GPU as a regular restaurant. It has a large kitchen with lots of chefs who can prepare a wide variety of dishes (tasks). It's good at handling many different types of orders, from appetizers to desserts. It's versatile but might take longer on very specialized or high-volume orders.
    *   **TPU: The Specialized Sushi Restaurant:** A TPU is like a sushi restaurant. It only focuses on making sushi and has specially trained chefs and specialized equipment to do it *very* quickly and efficiently. While it can't make pizza or pasta, it's the fastest option when you want a large amount of sushi (matrix multiplication - the core of deep learning).

    **Why it works:** This highlights the

In [25]:
response = model.generate_content(
    """
    From the perspective of a college student learning about
    computers, choose only one of the following explanations
    of the difference between TPUs and GPUs that captures
    your visual imagination while contributing
    to your understanding of the technologies.

    {brainstorm_response}
    """.format(brainstorm_response=brainstorm_response)
)

student_response = response.text

print(student_response)

I choose **The Factory Analogy (Intermediate Audience):**

*   **GPU: The Flexible Factory:** A GPU is a factory with a flexible production line. It can be reconfigured to produce different types of products (processes). It has general-purpose machines and can handle a wide range of manufacturing tasks. It's good for batch processing different types of jobs.
*   **TPU: The Assembly Line Factory:** A TPU is a factory with a fixed assembly line, specifically designed to produce one type of product (matrix multiplication). It is optimized for the repetitive and parallel operations required for deep learning. It is highly efficient at its dedicated task, but difficult to repurpose for other manufacturing processes.

**Why this analogy resonates with me and helps my understanding:**

As a college student, I'm beginning to understand the concept of parallel processing, and this analogy visualizes it perfectly. I imagine a large, sprawling factory (GPU) where different machines can be arrange

In [26]:
response = model.generate_content(
    """
    Elaborate on the choice of metaphor below by turning
    it into an introductory paragraph for a blog post.

    {student_response}
    """.format(student_response=student_response)
)

blog_post = response.text

print(blog_post)

Okay, here's a blog post introductory paragraph using the factory analogy, incorporating the reasoning you provided:

**The GPU vs. TPU Showdown: A Tale of Two Factories**

Imagine two factories, each churning out products, but operating under entirely different philosophies. One (the GPU) is a sprawling, adaptable space, a "flexible factory" where production lines can be reconfigured on the fly to manufacture a diverse range of goods. Think of it as a general-purpose machine shop, capable of handling countless manufacturing tasks. The other (the TPU) is a hyper-specialized "assembly line factory," meticulously designed to produce *one* thing – matrix multiplications – with unparalleled speed and efficiency. This distinction is key to understanding the difference between these powerful processors, and it's a visualization that really resonated with me as I began diving into the world of parallel processing. Just like how retooling our adaptable factory can be time-consuming, but enable

##The-End - Happy learning